# 언어 모델 finetuning


HuggingFace Trainer와 wandb를 합쳐 언어 모델이 글자를 생성하는 방법에 대해 알아봅시다.  
본 노트북에서는 자원의 제약때문에 작은 언어 모델(`TinyStories-33M`)을 사용할 것입니다.  
하지만 여기서 배운 내용은 당연히 큰 모델들에도 적용 가능합니다!

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import transformers
transformers.set_seed(42)

import wandb

In [ ]:
wandb.login(anonymous="allow")

In [ ]:
model_checkpoint = "roneneldan/TinyStories-33M"

### 데이터 준비하기

우선 던전과 드래곤 캐릭터의 이야기를 담은 데이터셋을 Huggingface로부터 불러옵시다

> 경고가 나타날수도 있긴 하지만 괜찮습니다 :)

In [ ]:
ds = load_dataset('MohamedRashad/characters_backstories')

In [ ]:
# 예시 하나를 살펴 보죠
ds["train"][400]

In [ ]:
# 이 데이터셋은 validation셋이 없으므로, train셋에서 20%를 validation셋으로 분리합니다.
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [ ]:
# 모델 체크포인트로부터 토크나이저를 생성합니다
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# 배치 내에서 동일한 시퀀스 길이를 갖기 위해 패딩을 사용합니다
tokenizer.pad_token = tokenizer.eos_token

# 텍스트와 타겟을 연결하여 토큰화하는 함수를 정의합니다
def tokenize_function(example):
    merged = example["text"] + " " + example["target"]
    batch = tokenizer(merged, padding='max_length', truncation=True, max_length=128)
    batch["labels"] = batch["input_ids"].copy()
    return batch

# 이 함수를 데이터셋에 적용하여 텍스트와 타겟 칼럼을 제거합니다
tokenized_datasets = ds.map(tokenize_function, remove_columns=["text", "target"])

In [ ]:
# 제대로 준비되었는지 확인해 봅시다
print(tokenizer.decode(tokenized_datasets["train"][900]['input_ids']))

### 학습
HF의 Transformer와 wandb 기능을 이용하여 사전학습된 모델을 우리 데이터셋에 finetuning 해봅시다

In [ ]:
# 사전학습된 체크포인트로부터 인과적(autoregressive) 언어 모델을 훈련시킬 것입니다
model = AutoModelForCausalLM.from_pretrained(model_checkpoint);

In [ ]:
# 새로운 wandb run 시작
run = wandb.init(project='dlai_lm_tuning', job_type="training", anonymous="allow")

In [ ]:
# 학습에 필요한 argument를 정의합니다
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-characters-backstories",
    report_to="wandb", # wandb에서 실험을 관리하기 위해서는 한 줄만 추가하면 됩니다
    num_train_epochs=1,
    logging_steps=1,
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    no_cuda=True, # cpu 사용을 강제합니다
)

In [ ]:
# HF의 Trainer를 사용합니다
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [ ]:
# 학습을 시작합니다
trainer.train()

In [ ]:
transformers.logging.set_verbosity_error() # 토크나이저 경고를 압축합니다

prefix = "Generate Backstory based on following information Character Name: "

prompts = [
    "Frogger Character Race: Aarakocra Character Class: Ranger Output: ",
    "Smarty Character Race: Aasimar Character Class: Cleric Output: ",
    "Volcano Character Race: Android Character Class: Paladin Output: ",
]

table = wandb.Table(columns=["prompt", "generation"])

for prompt in prompts:
    input_ids = tokenizer.encode(prefix + prompt, return_tensors="pt")
    output = model.generate(input_ids, do_sample=True, max_new_tokens=50, top_p=0.3)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    table.add_data(prefix + prompt, output_text)
    
wandb.log({'tiny_generations': table})

**주의**: LLM의 생성 결과는 항상 동일하지 않을수도 있습니다. 생성된 글자와 이야기는 영상에서와 다를 수 있습니다.

In [ ]:
wandb.finish()